In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import argparse



In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        logits = self.fc3(x)
        return logits

model = MLP().to(device)
print(model)



Using cpu device
MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)


In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
mnist_train = datasets.MNIST('./mnist', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST('./mnist', train=False, download=True, transform=transform)

batch_size = 32
train_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [4]:
def train(model, device=device, train_loader=train_loader, optimizer=optimizer, epochs=14, log_interval=10):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device=device, test_loader=test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
parser = argparse.ArgumentParser()
train(model, device, train_loader, optimizer)
test(model, device, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.324760
Train Epoch: 0 [320/60000 (1%)]	Loss: 2.307435
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.300279
Train Epoch: 0 [960/60000 (2%)]	Loss: 2.329796
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.363742
Train Epoch: 0 [1600/60000 (3%)]	Loss: 2.322748
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.335589
Train Epoch: 0 [2240/60000 (4%)]	Loss: 2.320942
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.304904
Train Epoch: 0 [2880/60000 (5%)]	Loss: 2.307624
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.298847
Train Epoch: 0 [3520/60000 (6%)]	Loss: 2.327993
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.320943
Train Epoch: 0 [4160/60000 (7%)]	Loss: 2.319995
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.328945
Train Epoch: 0 [4800/60000 (8%)]	Loss: 2.327298
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.335023
Train Epoch: 0 [5440/60000 (9%)]	Loss: 2.278872
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.337774
Train Epoch: 0 [6080/60000 (10%)]	Loss: 2.309146
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.30

KeyboardInterrupt: 